## Representação Genética e Função Fitness

Algoritmos Genéticos (GA) são técnicas de otimização inspiradas na
evolução natural. Cada solução candidata é representada por um
*cromossomo*, composto por *genes* que codificam os hiperparâmetros
do modelo.

Neste notebook são definidos:
- A representação genética dos hiperparâmetros;
- A função fitness;
- O método de avaliação do modelo.

### Representação Genética

Cada indivíduo da população representa uma configuração da Regressão Logística.

Cromossomo:
`[C, penalty, class_weight, max_iter]`

Onde cada posição do cromossomo representa um hiperparâmetro do modelo:

- `C`: valor numérico positivo que controla a força da regularização;
- `penalty`: tipo de regularização aplicada ao modelo (`l1` ou `l2`);
- `class_weight`: estratégia de balanceamento das classes (`None` ou `balanced`);
- `max_iter`: número máximo de iterações permitidas no treinamento.


### Codificação dos Genes

- Parâmetros contínuos (`C`) são representados como valores reais;
- Parâmetros discretos (`max_iter`) são representados como inteiros;
- Parâmetros categóricos (`penalty`, `class_weight`) são codificados
  por índices inteiros e decodificados durante a avaliação.

### Função de decodificação

In [1]:
def decode_individual(individual):
    return {
        "C": individual[0],
        "penalty": ["l1", "l2"][individual[1]],
        "class_weight": [None, "balanced"][individual[2]],
        "max_iter": individual[3]
    }

### Função Fitness

A função fitness avalia a qualidade de cada indivíduo com base no
desempenho do modelo treinado.

No contexto médico, métricas como Recall e F1-score são prioritárias,
pois reduzem o risco de falsos negativos.

### Implementação da função Fitness

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

def fitness_function(individual, X, y):
    params = decode_individual(individual)

    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    model = LogisticRegression(
        C=params["C"],
        penalty=params["penalty"],
        class_weight=params["class_weight"],
        max_iter=params["max_iter"],
        solver="liblinear"
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    return f1_score(y_val, y_pred)

- A função fitness definida neste notebook será utilizada diretamente no Algoritmo Genético implementado no próximo notebook